In [1]:
import sys
import os
import asyncio
import logging
from datetime import datetime

# Add the project root to Python path
sys.path.insert(0, os.path.abspath('..'))

from src.core.clob_client import PolymarketClient
from src.execution.order_tracker import OrderTracker
from py_clob_client.clob_types import OrderArgs

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

async def handle_order_filled(order):
    """Callback for when an order is filled"""
    print(f"\nOrder filled callback:")
    print(f"Order ID: {order.order_id}")
    print(f"Filled quantity: {order.filled_quantity}")
    print(f"Price: {order.price}")


# Initialize the Polymarket client
client = PolymarketClient()

# Get token IDs
token1_id = "43186308813322774389511655009441265359358460001746875331763940432725323754100"
token2_id = "42475543248404210065990230898512565125915441850041772863138114442518722564349"

# Get current market prices
token1_price = client.get_price(token1_id, "SELL")  # Best ask price for buying
token2_price = client.get_price(token2_id, "SELL")

print(f"\nToken 1 (YES) price: {token1_price}")
print(f"Token 2 (NO) price: {token2_price}")


PolymarketClient initialized with address: 0xA8fdCA71fa876ab477B3b5d4725292FFDacCd04F

Token 1 (YES) price: 0.58
Token 2 (NO) price: 0.43


In [5]:
from src.execution.order_executor import OrderExecutor

# Initialize order tracker and executor
executor = OrderExecutor()
order_tracker = OrderTracker(
    callback=handle_order_filled,
    executor=executor,
    status_check_interval=10,  # Check every 10 seconds
    cleanup_interval=60        # Cleanup every minute
)

# Create a signal for the order
signal = {
    "token_id": token1_id,
    "order_type": "limit",
    "side": "BUY",
    "quantity": 5.0,  # of shares
    "price": round(token1_price * 0.5, 2)  # Place order 5% below market
}

# Place the order using the executor
response = executor.execute_signal(signal)
print(response)


PolymarketClient initialized with address: 0xA8fdCA71fa876ab477B3b5d4725292FFDacCd04F
Placing LIMIT order: BUY 5.0 of 43186308813322774389511655009441265359358460001746875331763940432725323754100 at 0.29
{'errorMsg': '', 'orderID': '0xb36c883a68c0dfabf2e02c9e35520738c175bc33fd059ee363e983bfed7e2f1b', 'takingAmount': '', 'makingAmount': '', 'status': 'live', 'transactionsHashes': None, 'success': True}


In [7]:
if response and response.get("status") == "live" and response.get("success"):
    # Start tracking the order
    await order_tracker.track_order(
        order_id=response["orderID"],
        token_id=signal["token_id"],
        side=signal["side"],
        quantity=signal["quantity"],
        price=signal["price"],
        timeout_minutes=1
    )
    print(f"Started tracking order: {response['orderID']}")
else:
    print(f"Order placement failed or invalid response: {response}")

2025-03-19 23:24:29,319 - INFO - Started tracking order 0xb36c883a68c0dfabf2e02c9e35520738c175bc33fd059ee363e983bfed7e2f1b with 1 minute timeout


Started tracking order: 0xb36c883a68c0dfabf2e02c9e35520738c175bc33fd059ee363e983bfed7e2f1b


In [11]:
# Get all open orders
open_orders = executor.client.get_open_orders()

executor.cancel_all_orders()

Cancelling all orders.


{'not_canceled': {},
 'canceled': ['0xb36c883a68c0dfabf2e02c9e35520738c175bc33fd059ee363e983bfed7e2f1b']}